# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117985e+02     1.590923e+00
 * time: 0.3003361225128174
     1     1.069771e+01     8.717298e-01
 * time: 1.7437100410461426
     2    -1.164679e+01     9.936743e-01
 * time: 2.2585911750793457
     3    -3.385612e+01     7.778561e-01
 * time: 3.03285813331604
     4    -4.721433e+01     5.949236e-01
 * time: 3.790621042251587
     5    -5.686587e+01     2.016641e-01
 * time: 4.569359064102173
     6    -5.976898e+01     1.261770e-01
 * time: 5.096089124679565
     7    -6.089150e+01     5.426981e-02
 * time: 5.603931188583374
     8    -6.135051e+01     5.172242e-02
 * time: 6.136151075363159
     9    -6.161883e+01     3.354035e-02
 * time: 6.648972988128662
    10    -6.184550e+01     2.645759e-02
 * time: 7.163583040237427
    11    -6.200032e+01     2.434070e-02
 * time: 7.6688551902771
    12    -6.207016e+01     2.238588e-02
 * time: 8.19186019897461
    13    -6.212313e+01     1.573530e-02
 * time: 8.715403079986572
    14  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557753
    AtomicNonlocal      14.8522699
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336827

    total               -62.261666458357
